In [ ]:
from google.colab import drive

#you will have to change this for your own folder
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
labels_df = pd.read_csv('data/labels_imagenet.csv')
print(labels_df)

   label_id    object imagenet_label imagenet_id
0        l0    people         people   n07942152
1        l1     trees            NaN         NaN
2        l2     grass          grass   n12102133
3        l3  building       building   n02913152
4        l4      snow           snow   n11508382
..      ...       ...            ...         ...
87      l87     stage          stage   n04296562
88      l88      book           book   n07954211
89      l89     books            NaN         NaN
90      l90      flag           flag   n03354903
91      l91      pole           pole   n03976657

[92 rows x 4 columns]


In [ ]:
imagenet_labels_df = labels_df[~labels_df['imagenet_id'].isnull()]
print(imagenet_labels_df)

   label_id        object       imagenet_label imagenet_id
0        l0        people               people   n07942152
2        l2         grass                grass   n12102133
3        l3      building             building   n02913152
4        l4          snow                 snow   n11508382
6        l6         water                water   n07935504
7        l7          road                 road   n04096066
8        l8        person               person   n00007846
11      l11       bicycle              bicycle   n02834778
12      l12      pavement               paving   n14977504
16      l16  traffic sign          street sign   n06794110
20      l20        bridge               bridge   n02898711
22      l22          sand                 sand   n15019030
23      l23        street               street   n04334599
25      l25      graffiti             graffito   n03451798
27      l27          tree                 tree   n13104059
29      l29          wall                 wall   n045468

In [ ]:
synids = ' '.join(imagenet_labels_df['imagenet_id'].tolist())
print(synids)

n07942152 n12102133 n02913152 n11508382 n07935504 n04096066 n00007846 n02834778 n14977504 n06794110 n02898711 n15019030 n04334599 n03451798 n13104059 n04546855 n02828884 n03963645 n04951875 n02084071 n02792552 n04465501 n03637181 n03388043 n02958343 n09217230 n03327234 n02207449 n08256735 n02858304 n09917593 n03954731 n02121620 n03001627 n04350905 n04252225 n06793231 n03544360 n03904183 n02472293 n10289039 n13872592 n04379243 n13863020 n04296562 n07954211 n03354903 n03976657


In [ ]:
!python ./downloader.py -data_root ./data/images -use_class_list True -class_list {synids} -images_per_class 100

Picked the following clases:
['people', 'grass', 'building', 'snow', 'water', 'road', 'person', 'bicycle', 'paving', 'street sign', 'bridge', 'sand', 'street', 'graffito', 'tree', 'wall', 'bench', 'playground', 'sunniness', 'dog', 'barge', 'tractor', 'lamppost', 'fountain', 'car', 'beach', 'fence', 'worker', 'wedding', 'boat', 'child', 'plane', 'cat', 'chair', 'suit', 'snowplow', 'sign', 'house', 'pedestrian crossing', 'homo', 'man', 'cone', 'table', 'pencil', 'stage', 'book', 'flag', 'pole']
Scraping images for class "people"
Multiprocessing workers: 8
Scraping images for class "grass"
Multiprocessing workers: 8

Scraping stats:
STATS For class is_flickr:
 tried 242.0 urls with 171.0 successes
70.66115702479338% success rate for is_flickr urls 
0.05031138275101868 seconds spent per is_flickr succesful image download
STATS For class not_flickr:
 tried 0.0 urls with 0.0 successes
STATS For class all:
 tried 242.0 urls with 171.0 successes
70.66115702479338% success rate for all urls 
0.

In [ ]:
train_df = pd.read_csv('data/train_og.csv')
print(train_df)

       image_id                 labels
0    img256.jpg    l16 l18 l23 l25 l73
1     img98.jpg         l0 l24 l26 l39
2    img147.jpg                 l14 l3
3    img254.jpg             l2 l25 l29
4    img160.jpg      l0 l32 l38 l40 l6
..          ...                    ...
200  img105.jpg                    l51
201  img198.jpg      l0 l10 l11 l65 l7
202   img82.jpg                     l2
203  img124.jpg   l0 l1 l19 l22 l34 l6
204   img23.jpg  l1 l15 l18 l2 l21 l71

[205 rows x 2 columns]


In [ ]:
train_df_with_is_valid = train_df.assign(is_valid=True)
print(train_df_with_is_valid)

       image_id                 labels  is_valid
0    img256.jpg    l16 l18 l23 l25 l73      True
1     img98.jpg         l0 l24 l26 l39      True
2    img147.jpg                 l14 l3      True
3    img254.jpg             l2 l25 l29      True
4    img160.jpg      l0 l32 l38 l40 l6      True
..          ...                    ...       ...
200  img105.jpg                    l51      True
201  img198.jpg      l0 l10 l11 l65 l7      True
202   img82.jpg                     l2      True
203  img124.jpg   l0 l1 l19 l22 l34 l6      True
204   img23.jpg  l1 l15 l18 l2 l21 l71      True

[205 rows x 3 columns]


In [ ]:
new_train_df = train_df_with_is_valid
for label in imagenet_labels_df.itertuples(index=True):
  for image in os.listdir('data/images/imagenet_images/' + label.imagenet_label):
    data = {'image_id': 'imagenet_images/' + label.imagenet_label + "/" + image, 'labels': label.label_id, 'is_valid': False}
    new_train_df = new_train_df.append(pd.Series(data=data), ignore_index=True)

In [ ]:
all_label_ids = list(range(0, 92))
all_labels = list(map(lambda id: f"l{id}", all_label_ids))
data = {'image_id': 'dummy.jpg', 'labels': ' '.join(all_labels), 'is_valid': False}
new_train_df = new_train_df.append(pd.Series(data=data), ignore_index=True)

In [ ]:
new_train_df.to_csv('data/train_new.csv', index = False)